# Milestone 4
## COVID-19 and American Community Survey Data Exploration

### Part 1: Converting the Beam Pipelines to Dataflow

In [4]:
# clean up Census_refined.CensusLocation, make sure primary key is there and unique
%run CensusLocation_Dataflow.py

/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery_file_loads.py:900: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location


In [5]:
# clean JHU_refined.CovidLocation, make sure primary key is there and unique
%run CovidLocation_Dataflow.py

In [1]:
%run Overall_Dataflow.py

/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1971: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(
/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery_file_loads.py:900: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location


In [11]:
%run LocLinker_Dataflow.py

In [12]:
%run USCovidCases_Dataflow.py

### Part 2: Key Validation and ERD Update

In [1]:
%%bigquery
select count(Geo_ID), count(DISTINCT Geo_ID)
from Census_refined.CensusLocation_Dataflow

,f0_,f1_
0,52,52


In [2]:
%%bigquery
select count(Geo_ID), count(DISTINCT Geo_ID)
from Census_refined.Overall_Dataflow

,f0_,f1_
0,52,52


In [3]:
%%bigquery
select count(id), count(DISTINCT id)
from JHU_refined.CovidLocation_Dataflow

,f0_,f1_
0,52,52


In [4]:
%%bigquery
select count(Covid_ID) as covidCount, count(DISTINCT Covid_ID) as covidCountUnique, 
count(Geo_ID) as geoCount, count(DISTINCT Geo_ID) as geoCountUnique
from JHU_refined.LocLinker_Dataflow

,covidCount,covidCountUnique,geoCount,geoCountUnique
0,52,52,52,52


In [5]:
%%bigquery
select count(id), count(DISTINCT id)
from JHU_refined.USCovidCases_Dataflow

,f0_,f1_
0,10972,10972


As shown, all primary key values are unique in each table

In [6]:
%%bigquery
select Geo_ID
from Census_refined.CensusLocation_Dataflow
where Geo_ID not in
(select Geo_ID
from Census_refined.CensusLocation)

,Geo_ID


In [7]:
%%bigquery
select Geo_ID
from Census_refined.Overall_Dataflow
where Geo_ID not in
(select Geo_ID
from Census_refined.Overall)

,Geo_ID


In [8]:
%%bigquery
select id
from JHU_refined.CovidLocation_Dataflow
where id not in
(select id
from JHU_refined.CovidLocation)

,id


In [9]:
%%bigquery
select Covid_ID, Geo_ID
from JHU_refined.LocLinker_Dataflow
where Covid_ID not in
(select Covid_ID
from JHU_refined.LocLinker)
or Geo_ID not in
(select Geo_ID
from JHU_refined.LocLinker)

,Covid_ID,Geo_ID


In [10]:
%%bigquery
select id
from JHU_refined.USCovidCases_Dataflow
where id not in
(select id
from JHU_refined.USCovidCases)

,id


As shown above, no novel keys were found in child tables compared to the parent, tables, so the foreign keys maintain referential integrity.

ERD stored in final_unified_model.pdf

### Part 3: Cross-Dataset Queries

This view isolates mortality rate on the most recent update in the JHU_refined.USCovidCases_Dataflow table (11/09/2020) and joins with the Poverty and HealthWorker fields in the Census_refined.overall_Dataflow table based on State join in the linker table. The query then isolates data for states with a Health Worker Proportion below 6.

In [11]:
%%bigquery
create or replace view insights.v_State_Poverty_Mortality_Low_Healthforce as
select *
from (
select Province_State, Mortality_Rate, Poverty, HealthWorker
from JHU_refined.USCovidCases
join JHU_refined.LocLinker
on l_id = Covid_ID
join Census_refined.Overall
on LocLinker.Geo_ID = Overall.Geo_ID
order by Last_Update DESC, Testing_Rate DESC
limit 52 /*most recent data*/)
where HealthWorker < 6

""


This view extracts confirmed cases and mortality rate for all the records throughout the tracking period (4/19-11/09/20) in the JHU_refined.USCovidCases_Dataflow table for states with below average population uninsured rate and healthcare workforce proportion above 7 based on the Census_refined.overall_Dataflow table.

In [12]:
%%bigquery
create or replace view insights.v_History_Well_Prepared_States as
select Province_State, Confirmed, Mortality_Rate, Last_Update, HealthWorker, Insurance
from JHU_refined.USCovidCases
join JHU_refined.LocLinker
on l_id = Covid_ID
join 
(select * from Census_refined.Overall
where Insurance < (SELECT AVG(Insurance) FROM Census_refined.Overall)
and HealthWorker > 7) as t
on LocLinker.Geo_ID = t.Geo_ID
order by Last_Update DESC, HealthWorker DESC

""


This view extracts confirmed cases and mortality rate for all the records throughout the tracking period (4/19-11/09/20) in the JHU_refined.USCovidCases_Dataflow table for states with above average population uninsured rate and healthcare workforce proportion below 5.5 based on the Census_refined.overall_Dataflow table.

In [13]:
%%bigquery
create or replace view insights.v_History_Poorly_Prepared_States as
select Province_State, Confirmed, Mortality_Rate, Last_Update, HealthWorker, Insurance
from JHU_refined.USCovidCases
join JHU_refined.LocLinker
on l_id = Covid_ID
join 
(select * from Census_refined.Overall
where Insurance > (SELECT AVG(Insurance) FROM Census_refined.Overall)
and HealthWorker < 5.5) as t
on LocLinker.Geo_ID = t.Geo_ID
order by Last_Update DESC, HealthWorker DESC

""
